In [1]:
def generate_three_column_latex_table(kin2_path, kin3_path, kin4_path):
    def parse_file(path):
        with open(path, 'r') as f:
            lines = f.readlines()
        return {k.strip(): v.strip() for k, v in (line.split(',') for line in lines if ',' in line)}

    def valerr(val, err):
        return f"${val} \\pm {err}$"

    def round_str(x, digits=None):
        x = float(x)
        if digits is not None:
            return f"{round(x, digits):.{digits}f}"
        return f"{x}"

    d2, d3, d4 = map(parse_file, (kin2_path, kin3_path, kin4_path))

    latex_map = {
        "N_p": r"N_{p}", "N_n": r"N_{n}", "N_bg": r"N_{\text{bg}}", "N_total": r"N_{\text{total}}",
        "f_p": r"f_p", "f_bg": r"f_{\text{bg}}", "f_n": r"f_n", "f_nitrogen": r"f_{\text{nitrogen}}",
        "f_acc": r"f_{\text{acc}}",
        "A_acc": r"A_{\text{acc}}", "A_p": r"A_p", "A_inelastic": r"A_{\text{inelastic}}",
        "P_beam": r"P_{\text{beam}}", "P_target": r"P_{\text{target}}",
        "N_plus_raw": r"N_{\text{raw}}^+", "N_minus_raw": r"N_{\text{raw}}^-",
        "A_raw": r"A_{\text{raw}}", "A_Phys_wavg": r"A_{\text{phys}}",
        "tau": r"\tau", "epsilon": r"\epsilon", "Q2": r"Q^2", "Px": r"P_x", "Pz": r"P_z",
        "GE/GM * muN": r"\mu_n G_E/G_M"
    }

    rows = []

    def row(label, key, errkey=None, digits=None):
        def fmt(v, e=None):
            return valerr(v, e) if e is not None else f"${v}$"
        def get_vals(k, ek=None):
            if ek:
                return [(round_str(d[k], digits), round_str(d[ek], digits)) for d in (d2, d3, d4)]
            else:
                return [round_str(d[k], digits) for d in (d2, d3, d4)]

        if errkey:
            (v2, e2), (v3, e3), (v4, e4) = get_vals(key, errkey)
            rows.append(f"${label}$ & {fmt(v2,e2)} & {fmt(v3,e3)} & {fmt(v4,e4)} \\\\")
        else:
            v2, v3, v4 = get_vals(key)
            rows.append(f"${label}$ & ${v2}$ & ${v3}$ & ${v4}$ \\\\")

    # LaTeX header
    rows.extend([
        "\\begin{table}[h!]",
        "\\centering",
        "\\caption{Summary of extracted quantities for all three kinematics}",
        "\\resizebox{\\textwidth}{!}{%",
        "\\begin{tabular}{lccc}",
        "\\hline",
        "Quantity & Kin2 & Kin3 & Kin4 \\\\",
        "\\hline"
    ])

    # Round to 0 decimals
    for k in ['N_p', 'N_n', 'N_bg', 'N_total', 'N_plus_raw', 'N_minus_raw']:
        row(latex_map[k], k, digits=0)

    for k in ['f_p', 'f_bg', 'f_n', 'f_nitrogen','f_acc']:
        row(latex_map[k], k, k+'_e')

    for k in ['A_acc', 'A_p', 'A_inelastic']:
        row(latex_map[k], k, k+'_e')

    row(latex_map["P_beam"], "P_beam", "P_beam_e")
    row(latex_map["P_target"], "P_target", "P_target_e")

    # A_raw with rel error
    for d in [d2, d3, d4]:
        A = float(d["A_raw"])
        d["A_raw_err"] = f"{A * float(d['A_raw_rel_error']):.4f}"
    row(latex_map["A_raw"], "A_raw", "A_raw_err")

    # A_phys: two-line row
    def extract_phys_err(d):
        A = float(d["A_Phys_wavg"])
        return f"{A:.4f}", f"{A * float(d['A_phys_rel_sys_error']):.4f}", f"{A * float(d['A_phys_rel_sat_error']):.4f}"

    (v2, s2, t2), (v3, s3, t3), (v4, s4, t4) = map(extract_phys_err, (d2, d3, d4))
    rows.append(f"${latex_map['A_Phys_wavg']}$ & ${v2}$ & ${v3}$ & ${v4}$ \\\\")
    rows.append(f"& $\\pm {t2}$ (stat) $\\pm {s2}$ (sys) & $\\pm {t3}$ (stat) $\\pm {s3}$ (sys) & $\\pm {t4}$ (stat) $\\pm {s4}$ (sys) \\\\")

    for k in ['tau', 'epsilon', 'Q2', 'Px', 'Pz']:
        row(latex_map[k], k)

    # GE/GM with two-line
    ge_vals = [d["GE/GM * muN"] for d in (d2, d3, d4)]
    ges = [d["GE/GM_stat_error"] for d in (d2, d3, d4)]
    gesys = [d["GE/GM_sys_error"] for d in (d2, d3, d4)]
    rows.append(f"${latex_map['GE/GM * muN']}$ & ${ge_vals[0]}$ & ${ge_vals[1]}$ & ${ge_vals[2]}$ \\\\")
    rows.append(f"& $\\pm {ges[0]}$ (stat) $\\pm {gesys[0]}$ (sys) & $\\pm {ges[1]}$ (stat) $\\pm {gesys[1]}$ (sys) & $\\pm {ges[2]}$ (stat) $\\pm {gesys[2]}$ (sys) \\\\")

    # Footer
    rows.extend([
        "\\hline",
        "\\end{tabular}%",
        "}",
        "\\label{tab:kin_summary_all}",
        "\\end{table}"
    ])

    return "\n".join(rows)


In [2]:
latex_code = generate_three_column_latex_table("kin2results.txt", "kin3results.txt", "kin4combinedresults.txt")
print(latex_code)


\begin{table}[h!]
\centering
\caption{Summary of extracted quantities for all three kinematics}
\resizebox{\textwidth}{!}{%
\begin{tabular}{lccc}
\hline
Quantity & Kin2 & Kin3 & Kin4 \\
\hline
$N_{p}$ & $6280$ & $734$ & $226$ \\
$N_{n}$ & $134643$ & $16796$ & $3957$ \\
$N_{\text{bg}}$ & $9460$ & $8855$ & $3701$ \\
$N_{\text{total}}$ & $150690$ & $26836$ & $7822$ \\
$N_{\text{raw}}^+$ & $77550$ & $13884$ & $4161$ \\
$N_{\text{raw}}^-$ & $71380$ & $12656$ & $3853$ \\
$f_p$ & $0.0417 \pm 0.0005$ & $0.0273 \pm 0.001$ & $0.0289 \pm 0.0019$ \\
$f_{\text{bg}}$ & $0.0628 \pm 0.0007$ & $0.33 \pm 0.004$ & $0.4703 \pm 0.0094$ \\
$f_n$ & $0.8935 \pm 0.0034$ & $0.6259 \pm 0.0062$ & $0.5029 \pm 0.0098$ \\
$f_{\text{nitrogen}}$ & $0.0267 \pm 0.0005$ & $0.0244 \pm 0.0024$ & $0.0248 \pm 0.0004$ \\
$f_{\text{acc}}$ & $0.0147 \pm 0.0003$ & $0.0216 \pm 0.001$ & $0.0092 \pm 0.0011$ \\
$A_{\text{acc}}$ & $-0.0037 \pm 0.0058$ & $0.0068 \pm 0.0108$ & $0.0127 \pm 0.0529$ \\
$A_p$ & $0.001 \pm 0.0$ & $0.0002 \p

In [7]:
# Replace with your actual cut values
kin2 =  [0.07,1.28,-0.95,0.95,-0.70,0.70,126.00,131.00,0.025]
kin3 =  [-0.33,1.48,-0.50,0.50,-0.40,0.40,117.00,121.00,0.085]
kin4a = [-0.23,1.43,-0.30,0.30,-0.50,0.50,117.00,124.00,0.225]
kin4b = [-0.23,1.43,-0.25,0.25,-0.50,0.50,181.00,188.00,0.325]

# Label each kinematic and store all values together
kin_labels = ["Kin2", "Kin3", "Kin4a", "Kin4b"]
kin_data = [kin2, kin3, kin4a, kin4b]

cut_labels = ["W2", "dx", "dy", "Coincidence Time", "HCAL Energy"]
index_pairs = [(0,1), (2,3), (4,5), (6,7), (8,None)]

# Begin LaTeX table
print(r"\begin{table}[h!]")
print(r"\centering")
print("\\begin{tabular}{" + "|l|" + "c|c|" * len(kin_labels) + "}")
print(r"\hline")

# Header rows
header = r"\multirow{2}{*}{\textbf{Cut Variable}}"
for label in kin_labels:
    header += f" & \\multicolumn{{2}}{{c|}}{{\\textbf{{{label}}}}}"
print(header + r" \\")
print(r"\cline{2-" + str(1 + 2 * len(kin_labels)) + "}")
print(" & " + " & ".join(["Min & Max"] * len(kin_labels)) + r" \\")
print(r"\hline")

# Table body
for i, (cut_label, (i1, i2)) in enumerate(zip(cut_labels, index_pairs)):
    row = cut_label
    for kin in kin_data:
        val_min = f"{kin[i1]}"
        val_max = f"{kin[i2]}" if i2 is not None else "–"
        row += f" & {val_min} & {val_max}"
    print(row + r" \\")
    print(r"\hline")

# End LaTeX table
print(r"\end{tabular}")
print(r"\caption{Cut values for Kin2, Kin3, Kin4a, and Kin4b.}")
print(r"\label{tab:cut_values}")
print(r"\end{table}")


\begin{table}[h!]
\centering
\begin{tabular}{|l|c|c|c|c|c|c|c|c|}
\hline
\multirow{2}{*}{\textbf{Cut Variable}} & \multicolumn{2}{c|}{\textbf{Kin2}} & \multicolumn{2}{c|}{\textbf{Kin3}} & \multicolumn{2}{c|}{\textbf{Kin4a}} & \multicolumn{2}{c|}{\textbf{Kin4b}} \\
\cline{2-9}
 & Min & Max & Min & Max & Min & Max & Min & Max \\
\hline
W2 & 0.07 & 1.28 & -0.33 & 1.48 & -0.23 & 1.43 & -0.23 & 1.43 \\
\hline
dx & -0.95 & 0.95 & -0.5 & 0.5 & -0.3 & 0.3 & -0.25 & 0.25 \\
\hline
dy & -0.7 & 0.7 & -0.4 & 0.4 & -0.5 & 0.5 & -0.5 & 0.5 \\
\hline
Coincidence Time & 126.0 & 131.0 & 117.0 & 121.0 & 117.0 & 124.0 & 181.0 & 188.0 \\
\hline
HCAL Energy & 0.025 & – & 0.085 & – & 0.225 & – & 0.325 & – \\
\hline
\end{tabular}
\caption{Cut values for Kin2, Kin3, Kin4a, and Kin4b.}
\label{tab:cut_values}
\end{table}
